In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [2]:
df = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')

In [ ]:
df.shape

In [ ]:
df.head()

In [3]:
%pip install mlflow
%pip install dagshub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 26.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 63.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, auc, roc_curve, roc_auc_score
from sklearn.feature_selection import RFE
import xgboost as xgb
import mlflow
import mlflow.sklearn
import mlflow.xgboost
import dagshub
import warnings
import time
from scipy import stats
import category_encoders as ce

In [5]:
fraud = df['isFraud']
x = df.drop('isFraud', axis=1)

In [6]:
x_temp, x_test, y_temp, y_test = train_test_split(
    x, fraud, test_size=0.2, random_state=42, stratify=fraud
)

x_train, x_val, y_train, y_val = train_test_split(
    x_temp, y_temp, test_size=0.125, random_state=42, stratify=y_temp
)

# **Preprocess**

In [7]:
class Logistic_Regression_PrepData(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fill_with_mode(self, tmp, columns):
        for col in columns:
            mode_val = tmp[col].mode()[0]
            tmp[col] = tmp[col].fillna(mode_val)
        return tmp  

    def fill_with_median(self, tmp, columns):
        for col in columns:
            median_val = tmp[col].median()
            tmp[col] = tmp[col].fillna(median_val)
        return tmp  

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        if 'TransactionID' in X.columns:
            X = X.drop('TransactionID', axis=1)

        nullFraction = (X.isnull().sum() / len(X))
        removeCols = nullFraction[nullFraction >= 0.7].index
        X = X.drop(removeCols, axis=1)

        cat_col = X.select_dtypes(include=['object']).columns
        num_col = X.select_dtypes(include=['float64', 'int64']).columns

        X = self.fill_with_mode(X, cat_col)
        X = self.fill_with_median(X, num_col)

        return X

In [ ]:
prepper = PrepData()
x_train = prepper.transform(x_train)


In [ ]:
x_train.isnull().sum().sum()==0

# FE

In [8]:
class Logistic_Regression_FeatureEngineering(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.woe_enc = None
        self.cat_col = None

    def fit(self, X, y):
        self.cat_col = X.select_dtypes(include=['object']).columns
        self.woe_enc = ce.WOEEncoder(cols=self.cat_col)
        self.woe_enc.fit(X, y)
        return self

    def transform(self, X):
        return self.woe_enc.transform(X)

In [ ]:
fe = FeatureEngineering()

x_train_fe = fe.fit_transform(x_train, y_train)

In [ ]:
x_train_fe.shape

# Feature Selection **(Correlation and RFE)**

In [9]:
class Logistic_Regression_CorrelationSelector(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.05):
        self.threshold = threshold
        self.selected_features = None

    def fit(self, X, y):
        correlation_with_target = pd.DataFrame()
        for col in X.columns:
            correlation = np.abs(X[col].corr(y))
            correlation_with_target = pd.concat([
                correlation_with_target,
                pd.DataFrame({'Feature': [col], 'Correlation': [correlation]})
            ], ignore_index=True)
        
        correlation_with_target = correlation_with_target.sort_values('Correlation', ascending=False)
        self.selected_features = correlation_with_target[correlation_with_target['Correlation'] > self.threshold]['Feature'].tolist()
        return self

    def transform(self, X):
        return X[self.selected_features]

In [ ]:
selector = CorrelationFeatureSelector(threshold=0.05)

x_train_corr = selector.fit_transform(X_train_woe, y_train)


In [ ]:
x_train_corr.shape

In [ ]:
x_train_fe.shape

# Training

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Build pipeline
pipeline = Pipeline([
    ('prep', Logistic_Regression_PrepData()),
    ('feature_eng', Logistic_Regression_FeatureEngineering()),
    ('corr_selector', Logistic_Regression_CorrelationSelector(threshold=0.05)),
    ('model', LogisticRegression(max_iter=1000))
])
pipeline.fit(x_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('prep', Logistic_Regression_PrepData()),
                ('feature_eng', Logistic_Regression_FeatureEngineering()),
                ('corr_selector', Logistic_Regression_CorrelationSelector()),
                ('model', LogisticRegression(max_iter=1000))])

In [12]:

y_train_pred_proba = pipeline.predict_proba(x_train)[:, 1]
y_val_pred_proba = pipeline.predict_proba(x_val)[:, 1]

from sklearn.metrics import roc_auc_score

train_auc = roc_auc_score(y_train, y_train_pred_proba)
val_auc = roc_auc_score(y_val, y_val_pred_proba)

print(f"Train AUC: {train_auc:.4f}")
print(f"Validation AUC: {val_auc:.4f}")


Train AUC: 0.7998
Validation AUC: 0.8057


In [13]:
y_pred_proba = pipeline.predict_proba(x_test)[:, 1]

test_auc = roc_auc_score(y_test, y_pred_proba)

print(f"Test ROC AUC without GridSearch: {test_auc:.4f}")


Test ROC AUC without GridSearch: 0.8038


In [10]:
import dagshub
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

dagshub.init(repo_owner='goguaD', repo_name='Fraud-Detection', mlflow=True)
mlflow.set_experiment("LogisticRegression_Training")

with mlflow.start_run(run_name="LogisticRegression_Feature_Selection"):
    pipeline = Pipeline([
        ('prep', Logistic_Regression_PrepData()),
        ('feature_eng', Logistic_Regression_FeatureEngineering()),
        ('corr_selector', Logistic_Regression_CorrelationSelector(threshold=0.05)),
        ('model', LogisticRegression(max_iter=1000))
    ])
    
    pipeline.fit(x_train, y_train)
    
    y_test_pred_proba = pipeline.predict_proba(x_test)[:, 1]
    auc = roc_auc_score(y_test, y_test_pred_proba)
    
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_param("feature_selection", "Correlation")
    mlflow.log_param("max_iter", 1000)
    
    mlflow.log_metric("Test ROC AUC", auc)
    
    mlflow.sklearn.log_model(pipeline, artifact_path="pipeline_model")

print(f"Logged Test AUC: {auc:.4f}")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=ed406c61-6868-4244-88dd-78478b8fc7a6&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=315132a0e1d8e6c84d35e4a9ee90e7d67ad33fde5d0eeeb06a64aae65d8b3e4d




Accessing as goguaD

Initialized MLflow to track repo "goguaD/Fraud-Detection"

Repository goguaD/Fraud-Detection initialized!

2025/04/27 11:49:29 INFO mlflow.tracking.fluent: Experiment with name 'LogisticRegression_Training' does not exist. Creating a new experiment.
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2025/04/27 11:51:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LogisticRegression_Feature_Selection at: https://dagshub.com/goguaD/Fraud-Detection.mlflow/#/experiments/2/runs/5fd179d50bcc480fa21489e92b0a9142
🧪 View experiment at: https://dagshub.com/goguaD/Fraud-Detection.mlflow/#/experiments/2
Logged Test AUC: 0.8038
